In [14]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import json

In [15]:

path_02 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_02/data/*.png"
path_03 = "/Users/mathildevangkilde/Desktop/34759_final_project_raw/calib/image_03/data/*.png"
images_02 = sorted(glob.glob(path_02))
images_03 = sorted(glob.glob(path_03))

# for at finde flere chessboards på et billede, laver vi bounding boxes omkring hvert chessboard således at 
# hver roi kun har ét chessboard i sig, som vi så kan køre findChessboardCornersSB på. 
with open("rois_02.json", "r") as f:
    rois_02 = json.load(f)[:13]

with open("rois_03.json", "r") as f:
    rois_03 = json.load(f)[:13]

# chessboard mønstre 
patterns = [
    {"pattern_size": (11, 7), "square_size": 0.010},   # C
    {"pattern_size": (7, 5),  "square_size": 0.010},   # A
    {"pattern_size": (5, 7),  "square_size": 0.010},   # B
    {"pattern_size": (5, 15), "square_size": 0.010},   # D
]

# hvert roi indeholder et chessboard med et af de fire mønstre 
roi_patterns = [0,1,2,3,0,1,2,3,0,1,2,3,0]  

# Subpixel refinement criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objpoints = []
imgpoints_02 = []
imgpoints_03 = []

# looper over alle 19 par med 1 billede fra 02, og 1 billede fra 03
for f02, f03 in zip(images_02, images_03):
    gray02 = cv2.cvtColor(cv2.imread(f02), cv2.COLOR_BGR2GRAY)
    gray03 = cv2.cvtColor(cv2.imread(f03), cv2.COLOR_BGR2GRAY)

    corners02_img = []
    corners03_img = []
    objpoints_img = []

    # Looper over rois
    for idx, (roi02, roi03) in enumerate(zip(rois_02, rois_03)):
        pat = patterns[roi_patterns[idx]]
        ps = pat["pattern_size"]
        sq = pat["square_size"]

        # Cropper rois 
        x1,y1,x2,y2 = roi02
        x1b,y1b,x2b,y2b = roi03
        crop02 = gray02[y1:y2, x1:x2]
        crop03 = gray03[y1b:y2b, x1b:x2b]

        ok02, c02 = cv2.findChessboardCornersSB(crop02, ps) #finder chessboard i hvert roi 
        ok03, c03 = cv2.findChessboardCornersSB(crop03, ps)

        if not (ok02 and ok03):
            continue  # skip if board not found in both cameras, det er vigtigt vi finder det samme board i begge billeder i parret 

        # corner refining tingting 
        c02 = cv2.cornerSubPix(crop02, c02, (11,11), (-1,-1), criteria)
        c03 = cv2.cornerSubPix(crop03, c03, (11,11), (-1,-1), criteria)

        c02_full = c02 + np.array([[x1, y1]], dtype=np.float32)
        c03_full = c03 + np.array([[x1b, y1b]], dtype=np.float32)

        objp = np.zeros((ps[0]*ps[1],3), np.float32) #object point til camera calibration for hvert board 
        objp[:,:2] = np.mgrid[0:ps[0], 0:ps[1]].T.reshape(-1,2)
        objp *= sq

        corners02_img.append(c02_full.reshape(-1,2))
        corners03_img.append(c03_full.reshape(-1,2))
        objpoints_img.append(objp)

    if len(corners02_img) > 0:
        imgpoints_02.append(np.vstack(corners02_img)) # vi bruger kun billeder hvor mindst et skakbræt blev fundet i begge par 
        imgpoints_03.append(np.vstack(corners03_img))
        objpoints.append(np.vstack(objpoints_img))

print(f"Collected {len(objpoints)} valid image pairs for calibration.")



Collected 19 valid image pairs for calibration.


In [16]:
# Camera calibration for kamera 02 og 03 
ret02, mtx02, dist02, rvecs02, tvecs02 = cv2.calibrateCamera(
    objpoints, imgpoints_02, gray02.shape[::-1], None, None
)
ret03, mtx03, dist03, rvecs03, tvecs03 = cv2.calibrateCamera(
    objpoints, imgpoints_03, gray03.shape[::-1], None, None
)

In [ ]:
# resultaterne er sådan vildt dårlige så vi matcher efter keypoints og descriptors i boksene også 
def extract_keypoints_from_boxes(img, bboxes, max_features=50):
    keypoints_all = []
    descriptors_all = []
    sift = cv2.SIFT_create(max_features)

    for box in bboxes:
        x_min, y_min, x_max, y_max = box
        roi = img[y_min:y_max, x_min:x_max]
        kp, des = sift.detectAndCompute(roi, None)
        if kp is not None:
            for k in kp:
                k.pt = (k.pt[0] + x_min, k.pt[1] + y_min)
            keypoints_all.extend(kp)
            descriptors_all.extend(des)
    
    descriptors_all = np.array(descriptors_all, dtype=np.float32) if descriptors_all else None
    return keypoints_all, descriptors_all

img02_sample = cv2.imread(images_02[0], cv2.IMREAD_GRAYSCALE) #gør det bare lige med det første par, følger en lokal reprojection error er fint når de basically er det samme billede 
img03_sample = cv2.imread(images_03[0], cv2.IMREAD_GRAYSCALE)

bboxes02 = rois_02
bboxes03 = rois_03

kp02, des02 = extract_keypoints_from_boxes(img02_sample, bboxes02) # keypoints til bounding boxes i kp02 og kp03
kp03, des03 = extract_keypoints_from_boxes(img03_sample, bboxes03)

# matcher descriptorer 
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(des02, des03)
matches = sorted(matches, key=lambda x: x.distance)

pts02 = np.array([kp02[m.queryIdx].pt for m in matches])
pts03 = np.array([kp03[m.trainIdx].pt for m in matches])

pts02_norm = cv2.undistortPoints(np.expand_dims(pts02, axis=1), mtx02, dist02) #normalizerer punkterne 
pts03_norm = cv2.undistortPoints(np.expand_dims(pts03, axis=1), mtx03, dist03)

E, mask = cv2.findEssentialMat(pts02_norm, pts03_norm, method=cv2.RANSAC, prob=0.999, threshold=1.0) #RANSAC  

# reprojection error 
errors = []
for i in range(len(pts02_norm)):
    if mask[i]:
        x1 = np.array([pts02_norm[i][0][0], pts02_norm[i][0][1], 1.0])
        x2 = np.array([pts03_norm[i][0][0], pts03_norm[i][0][1], 1.0])
        errors.append(abs(x2.T @ E @ x1))

errors = np.array(errors)
print("Mean |x2^T E x1| error:", errors.mean())
print("Std  |x2^T E x1| error:", errors.std())


Mean |x2^T E x1| error: 0.25796930037084703
Std  |x2^T E x1| error: 0.7135773376141629


In [19]:
# image size og criteria til stereo calibration
image_size = (gray02.shape[1], gray02.shape[0])
criteria_stereo = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)

# Stereo calibration, finder camera extrinsics (rotation R og translation T)
ret_stereo, mtx02, dist02, mtx03, dist03, R, T, E, F = cv2.stereoCalibrate(
    objpoints,
    imgpoints_02,
    imgpoints_03,
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    criteria=criteria_stereo,
    flags=cv2.CALIB_FIX_INTRINSIC
)

print("Rotation:\n", R)
print("Translation:\n", T)

Rotation:
 [[ 0.96638753 -0.25683202 -0.01150867]
 [ 0.24064176  0.91940828 -0.31109477]
 [ 0.09048027  0.29786864  0.95030921]]
Translation:
 [[-0.00104905]
 [ 0.01786891]
 [ 0.00944807]]


In [22]:
# stereo rectification 
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    mtx02, dist02,
    mtx03, dist03,
    image_size,
    R, T,
    flags=cv2.CALIB_ZERO_DISPARITY,
    alpha=1  # 0 = zoomer, 1 = fuldt billede
)

print("Rectification matrix R1:\n", R1)
print("Rectification matrix R2:\n", R2)
print("Projection matrix P1:\n", P1)
print("Projection matrix P2:\n", P2)
print("Disparity-to-depth matrix Q:\n", Q)

# Compute undistort/rectify maps (for later use on any stereo pair)
map_x_02, map_y_02 = cv2.initUndistortRectifyMap(
    mtx02, dist02, R1, P1, image_size, cv2.CV_32FC1
)
map_x_03, map_y_03 = cv2.initUndistortRectifyMap(
    mtx03, dist03, R2, P2, image_size, cv2.CV_32FC1
)



Rectification matrix R1:
 [[ 0.97866622 -0.20478935 -0.01654559]
 [ 0.20459733  0.96405002  0.16955083]
 [-0.01877143 -0.16931885  0.98538255]]
Rectification matrix R2:
 [[ 0.99855771  0.05237019  0.01182623]
 [-0.05182991  0.88284386  0.466798  ]
 [ 0.01400558 -0.4667377   0.88428489]]
Projection matrix P1:
 [[-5.67779408e+03  0.00000000e+00 -1.02991715e+02  0.00000000e+00]
 [ 0.00000000e+00 -5.67779408e+03  4.23368892e+03  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Projection matrix P2:
 [[-5.67779408e+03  0.00000000e+00 -1.02991715e+02  0.00000000e+00]
 [ 0.00000000e+00 -5.67779408e+03  4.23368892e+03 -1.14919500e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]
Disparity-to-depth matrix Q:
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  1.02991715e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -4.23368892e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.67779408e+03]
 [ 0.00000000e+00  0.00000000e+

In [23]:
#formatterer resultaterne så vi kan sammenligne med deres rect 
# formatterer til pretty print matrice/vektor 

def print_matrix(name, M): 
    M_flat = M.reshape(-1)
    print(f"{name}: " + " ".join(f"{v:.6e}" for v in M_flat))

def print_vector(name, v):
    v_flat = v.reshape(-1)
    print(f"{name}: " + " ".join(f"{x:.6e}" for x in v_flat))

print("\nKamera 02 resultater")

#kamera 02 
print("S_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_02", mtx02)
print_vector("D_02", dist02)

# stereo R,T ransformen fra 02 til 03
print_matrix("R_02", np.eye(3))
print_vector("T_02", np.zeros((3,1)))

print("\nRectified resultater for 02")
print("S_rect_02:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_02", R1)
print_matrix("P_rect_02", P1)


print("\nKamera 03 resultater")

print("S_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("K_03", mtx03)
print_vector("D_03", dist03)

# kamera 03 pose ift kamera 02, fra stereoCalibrate
print_matrix("R_03", R)
print_vector("T_03", T)

print("\nRectified resultater for 03")
print("S_rect_03:", f"{image_size[0]:.6e}", f"{image_size[1]:.6e}")
print_matrix("R_rect_03", R2)
print_matrix("P_rect_03", P2)

print("\n Q matrice:")
print_matrix("Q", Q)


Kamera 02 resultater
S_02: 1.392000e+03 5.120000e+02
K_02: 4.901113e+02 0.000000e+00 6.954994e+02 0.000000e+00 3.523937e+01 2.555000e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_02: -1.714888e-02 8.022426e-05 5.425589e-03 9.933748e-04 -1.062708e-07
R_02: 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00
T_02: 0.000000e+00 0.000000e+00 0.000000e+00

Rectified resultater for 02
S_rect_02: 1.392000e+03 5.120000e+02
R_rect_02: 9.786662e-01 -2.047893e-01 -1.654559e-02 2.045973e-01 9.640500e-01 1.695508e-01 -1.877143e-02 -1.693188e-01 9.853825e-01
P_rect_02: -5.677794e+03 0.000000e+00 -1.029917e+02 0.000000e+00 0.000000e+00 -5.677794e+03 4.233689e+03 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00

Kamera 03 resultater
S_03: 1.392000e+03 5.120000e+02
K_03: 3.783728e+02 0.000000e+00 6.955000e+02 0.000000e+00 2.854406e+01 2.555000e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_03: -2.478044e-03 1.005276e-06 -2.2